# Project 3 - Solving Linear PDE's using Physics Informed Neural Network (PINN)
## The differential equations
### Diffusion equation
$$\frac{\partial u}{\partial t} - D\frac{\partial^{2}}{\partial x^{2}} = f(t,x) $$
with $D$ as the diffusion coefficient, and $f(t,x)$ as a generic source function. 

### Burgers equation
$$ \frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} - \mu\frac{\partial^{2} u}{\partial x^{2}} = f(t,x) $$
Here, $\mu$ denotes the viscosity of the system.

### Wave equation
$$\frac{\partial^{2} u}{\partial t^{2}} - c^{2}\frac{\partial^{2} u}{\partial x^{2}} = f(t,x) $$
with $c$ denoting the wave speed.

## Solutions to PDE's using a PINN
This part solves the equations below using two PINN-implementations.<br />

The first is an implementation uses Autograd's differentiation method...  using a method defining a trial function...<br />

The second implementation uses TensorFlow's Keras-API to build out the network and perform the feed-forward and backpropagation part, and is based on an implementation by ... ([PINNs-TF2](https://github.com/pierremtb/PINNs-TF2.0?tab=readme-ov-file)). This uses...

Based on implementation from ... 

#### Program imports and defaults

In [ ]:
from PDEq import *
from support import *
from network import *
from networkFlowTorch import *

from time import time

import autograd.numpy as anp
from autograd import elementwise_grad

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, KFold, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler

## Random seed
default_seed = 15; anp.random.seed(default_seed)

## Figure defaults
plt.rcParams["figure.figsize"] = (8,3); plt.rcParams["font.size"] = 10


## Problem setup

In [ ]:
## Differential equations
test_cases = ['diff1d','burgers1d','wave1d']
test = test_cases[2]

## Coefficients
c,D,amplitude = 1.,1.,1.

if test == 'diff1d':
    PDE = Diffusion1D(sim_type='own',amp=amplitude,D=D)
    PDE_tf = Diffusion1D(sim_type='flow',amp=amplitude,D=D)
    t0,tN,x0,xN = 0,1,0,1
elif test == 'burgers1d':
    PDE = Burger1D(sim_type='own')
    PDE_tf = Burger1D(sim_type='flow')
    t0,tN,x0,xN = 0,1,-1,1
elif test == 'wave1d':
    PDE = Wave1D(sim_type='own',amp=amplitude,c=c)
    PDE_tf = Wave1D(sim_type='flow',amp=amplitude,c=c)
    t0,tN,x0,xN = 0,1,-1,1

## Defining the source function (necessary?)
f    = PDE.right_hand_side
f_tf = PDE_tf.right_hand_side

## Domain setup
Nt,Nx = 10,10

t_bound = [t0,tN]
x_bound = [x0,xN]

t = np.linspace(t_bound[0],t_bound[1],Nt)
x = np.linspace(x_bound[0],x_bound[1],Nx)
        
domain_array = np.array([t,x])

### Network configuration

In [ ]:
## Layer configuration
layer_out_sizes = [20,20,20,20,20,20,1]
#layer_out_sizes = [20,20,20,1]

## Activation functions
hidden_func_tf = 'tanh' # sigmoid, relu, elu, leaky_relu, tanh, swish, gelu, hard_sigmoid, exponential
hidden_func = tanh
hidden_der = elementwise_grad(hidden_func,0)

act_funcs_tf = []; act_funcs = []; act_ders = []
for i in range(len(layer_out_sizes)-1):
    act_funcs_tf.append(hidden_func_tf)
    act_funcs.append(hidden_func)
    act_ders.append(hidden_der)

## Output layer activation function set to identity
act_funcs_tf.append(None); 

act_funcs.append(identity); 
output_der = identity #elementwise_grad(act_funcs[-1],0);
act_ders.append(output_der)

## Network setup using the TensorFlow-implementation

In [ ]:
## Gradient descent method, regularizer and learning rate
lmbda_tf = 1e-2
gd_method_tf = 'adam'
eta_tf = 1e-2 # None
tf_epoch = 100

## Collocation tensor parameters
c_points = 10000; b_points = 100; i_points = 100

## Network initializtion
TFNetwork = FFNNetworkFlow(layer_output_size=layer_out_sizes,
                           activation_functions=act_funcs_tf,
                           PDE=PDE_tf,
                           source_function=f_tf,
                           domain_array=domain_array,
                           domain=x_bound,
                           gd_method=gd_method_tf,
                           learning_rate=eta_tf)

## Setup of collocation tensor
TFNetwork.collocation_setup(bounds=(x_bound,t_bound), colloc_points=c_points,
                            bound_points=b_points, init_points=i_points)
TFNetwork.create_layers(lmbda=lmbda_tf)

### Network setup using `FFNNetwork`-class

In [ ]:
## Gradient descent method, regularizer and learning rate
eta = [eta_tf if eta_tf is not None else 1e-4]
lmbda = lmbda_tf
epoch = tf_epoch #100
gd_method = ADAM(learning_rate=eta,lmbda=lmbda)

## Network initializtion
OwnNetwork = FFNNetwork(layer_output_size=layer_out_sizes,
                        activation_functions=act_funcs,
                        activation_derivatives=act_ders,
                        PDE=PDE,
                        source_function=f,
                        domain_array=domain_array,
                        domain=x_bound,
                        random_state=default_seed)

### Training the networks

In [ ]:
epoch = epoch
tf_epoch = tf_epoch

own_net_timer = [time()]
P = OwnNetwork.train_network(GDMethod=gd_method,learn_rate=eta,epochs=epoch)
own_net_timer.append(time())


tf_net_timer = [time()]
TFNetwork.train_network(epochs=tf_epoch)
tf_net_timer.append(time())

print('Plain PINN-solver time     : %.2f sec.' %(own_net_timer[1]-own_net_timer[0]))
print('TensorFlow PINN-solver time: %.2f sec.' %(tf_net_timer[1]-tf_net_timer[0]))

### Plotting results from networks
#### Plain implementation

In [ ]:
f_names = [str(test)+'_own_network.png',str(test)+'_analytic.png',str(test)+'_difference.png']
OwnNetwork.plot_result(save=False)

#### TensorFlow implementation

In [ ]:
f_names = [str(test)+'_tf_network.png',str(test)+'_analytic.png',str(test)+'_tf_difference.png']
TFNetwork.plot_results(save=False,f_name=f_names)

## Finite difference solutions
This part solves the same PDE's using the _finite difference_ (FD)-method. The implementation is based on classes and methods from the lecture notes and exercises of the UiO-course _MAT-MEK4270 Numerical methods for partial differential equations_ ([MAT-MEK4270](https://matmek-4270.github.io/matmek4270-book/intro.html))

In [ ]:
from FiniteDiff import *
import sympy as sp

x_s,t_s,c_s,L_s = sp.symbols('x, t, c, L')

## Domain setup
Nt,Nx = 10,10#Nt,Nx

## Simulation and equation coefficients
cfl = 0.01
c,D,amplitude = 1., D, amplitude

test='wave1d'
if test == 'diff1d':
    x_bound = [0,1]
    bc = {'left': 0, 'right': 0}
    u0 = amplitude * sp.sin(sp.pi*x_s/L_s)
    fd_solver = Diffusion1DSolver(N=Nx,domain=x_bound,cfl=cfl,u0=u0,D=D)
    
elif test == 'wave1d':
    x_bound = [-1,1]
    bc = {'left': 0, 'right': 0}
    u0 = amplitude * sp.sin(sp.pi*x_s/L_s)
    fd_solver = Wave1DSolver(N=Nx,domain=x_bound,cfl=cfl,u0=u0,amp=amplitude,c=c)


### Solving the equations with the FD-method

In [ ]:
fd_timer = [time()]
fd_solver.solver(tN=t_bound[1],cfl=cfl,bc=bc,ic=0)
fd_solver.solver(tN=1,cfl=cfl,bc=bc,ic=0)

fd_timer.append(time())

print('FD-solver time: %.2f sec.' %(fd_timer[1]-fd_timer[0]))

### Plotting FD - results

In [ ]:
f_names = [str(test)+'_fd_solution.png',str(test)+'_analytic.png',str(test)+'_fd_difference.png']
fd_solver.plot_result(tN=t_bound[1],save=False,f_name=f_names)

### Comparison between the PINN and FD-methods